## Attributes and Properties

In [53]:
class OperatorExpression(object):
    """Expression of the form a <operator> b, where <operator> is
    +, -, *, /, //, ** or %"""
    
    def __init__(self, root, left, right):
        self.contents = (root, left, right)
        
    def prefixForm(self):
        "Method: returns string in the form root(left, right)"
        root, left, right = self.contents[0], self.contents[1], \
self.contents[2]
        return root+'('+str(left)+', '+str(right)+')'
    
    def infixOperator(self):
        """Method: returns the infix operator 
        associated with the root node"""
        root = self.contents[0]
        if root == 'plus':
            return '+'
        elif root == 'subtract':
            return '-'
        elif root == 'times':
            return '*'
        elif root == 'divide':
            return '/'
        elif root == 'intdivide':
            return '//'
        elif root == 'power':
            return '**'
        elif root == 'mod':
            return '%'
    
    def infixForm(self):
        """Method: returns string in the form (left) op (right) where 
        op is the infix operator associated with the root node"""
        root, left, right = self.contents[0], self.contents[1], \
self.contents[2]
        return \
    '('+str(left)+') '+self.infixOperator()+' ('+str(right)+')'
    
    def subs(self, var, val):
        """Method: returns an OperatorExpression 
        with var substituted for val"""
        root, left, right = self.contents[0], self.contents[1], \
self.contents[2]
        if left == var and right == var:
            # variable present on both sides of the expression
            return OperatorExpression(root, val, val)
        elif left == var:
            # variable present on the left hand side only
            return OperatorExpression(root, val, right)
        elif right == var:
            # variable present on the right hand side only
            return OperatorExpression(root, left, val)
        else:
            # variable not present
            return self

    def evaluate(self):
        "Method: returns the numerical value of an OperatorExpression"
        root, left, right = self.contents[0], self.contents[1], \
self.contents[2]
        if root == 'plus':
            return left + right
        elif root == 'subtract':
            return left - right
        elif root == 'times':
            return left * right
        elif root == 'divide':
            return left / right
        elif root == 'intdivide':
            return left // right
        elif root == 'power':
            return left ** right
        elif root == 'mod':
            return left % right

# cell for the new class, complete with the infixForm,
# subs and evaluate methods
class ExpressionTree(OperatorExpression):
    "A compound tree built from OperatorExpressions"
    ### BEGIN SOLUTION
    
    def __init__(self, root, left=None, right=None):
        if left==None or right==None:
            # default: childless tree, contents consist of a 1-tuple
            self.contents = (root,)
        else:
            # parent tree, contents consist of a 3-tuple
            self.contents = (root, left, right)

    def prefixForm(self):
        "Method: returns string consisting of nested expressions"
        if len(self.contents) == 1:
            # childless tree: return the root node as a string
            return str(self.contents[0])
        else:
            # parent tree: recurse down the structure
            root, left, right = self.contents[0], self.contents[1], \
self.contents[2]
            return \
        root+'('+left.prefixForm()+', '+right.prefixForm()+')'       

    def infixForm(self):
        "Method: returns string consisting of nested expressions in infix form"
        if len(self.contents) == 1:
            return str(self.contents[0])
        else:
            root, left, right = self.contents[0], self.contents[1], \
self.contents[2]
            return '('+left.infixForm()+') '+self.infixOperator()+' ('+\
right.infixForm()+')' 
        
    def subs(self, var, val):
        """Method: returns an ExpressionTree 
        with var substituted for val"""
        if len(self.contents) == 1:
            if self.contents[0] == var:
                return ExpressionTree(val)
            else:
                return self
        else:
            root, left, right = self.contents[0], self.contents[1], \
self.contents[2]
            return ExpressionTree(root, left.subs(var, val), \
right.subs(var, val))

    def evaluate(self):
        "Method: returns the numerical value of an ExpressionTree"
        if len(self.contents) == 1:
            if isinstance(self.contents[0], str):
                return eval(self.contents[0])
            else:
                return self.contents[0]
        else:
            root, left, right = self.contents[0], self.contents[1], \
self.contents[2]
            if root == 'plus':
                return left.evaluate() + right.evaluate()
            elif root == 'subtract':
                return left.evaluate() - right.evaluate()
            elif root == 'times':
                return left.evaluate() * right.evaluate()
            elif root == 'divide':
                return left.evaluate() / right.evaluate()
            elif root == 'intdivide':
                return left.evaluate() // right.evaluate()
            elif root == 'power':
                return left.evaluate() ** right.evaluate()
            elif root == 'mod':
                return left.evaluate() % right.evaluate()    
    
    ### END SOLUTION

In [59]:
expr1 = ExpressionTree('x')
expr2 = ExpressionTree(1)
expr3 = ExpressionTree('times', expr1, expr2)

In [60]:
expr3.infixForm()

'(x) * (1)'

In [61]:
# pre-simplification by re-writing init
class ExpressionTree(OperatorExpression):
    "A compound tree built from OperatorExpressions"
    
    def __init__(self, root, left=None, right=None):
        if left==None or right==None:
            # default: childless tree, contents consist of a 1-tuple
            self.contents = (root,)
        elif (
            root=='plus' and right.contents==(0,) or
            root=='subtract' and right.contents==(0,) or
            root=='times' and right.contents==(1,) or
            root=='power' and right.contents==(1,) or
            root=='intdivide' and right.contents==(1,)
        ):
            # simplify to left contents
            self.contents = left.contents
        elif (
            root=='plus' and left==0 or
            root=='times' and left==1
        ):
            # simplify to right contents
            self.contents = right.contents
        elif root=='power' and left==1:
            # simplify to 1
            self.contents = (1,)
        else:
            # parent tree, contents consist of a 3-tuple
            self.contents = (root, left, right)

    def prefixForm(self):
        "Method: returns string consisting of nested expressions"
        if len(self.contents) == 1:
            # childless tree: return the root node as a string
            return str(self.contents[0])
        else:
            # parent tree: recurse down the structure
            root, left, right = self.contents[0], self.contents[1], \
self.contents[2]
            return \
        root+'('+left.prefixForm()+', '+right.prefixForm()+')'       

    def infixForm(self):
        "Method: returns string consisting of nested expressions in infix form"
        if len(self.contents) == 1:
            return str(self.contents[0])
        else:
            root, left, right = self.contents[0], self.contents[1], \
self.contents[2]
            return '('+left.infixForm()+') '+self.infixOperator()+' ('+\
right.infixForm()+')' 
        
    def subs(self, var, val):
        """Method: returns an ExpressionTree 
        with var substituted for val"""
        if len(self.contents) == 1:
            if self.contents[0] == var:
                return ExpressionTree(val)
            else:
                return self
        else:
            root, left, right = self.contents[0], self.contents[1], \
self.contents[2]
            return ExpressionTree(root, left.subs(var, val), \
right.subs(var, val))

    def evaluate(self):
        "Method: returns the numerical value of an ExpressionTree"
        if len(self.contents) == 1:
            if isinstance(self.contents[0], str):
                return eval(self.contents[0])
            else:
                return self.contents[0]
        else:
            root, left, right = self.contents[0], self.contents[1], \
self.contents[2]
            if root == 'plus':
                return left.evaluate() + right.evaluate()
            elif root == 'subtract':
                return left.evaluate() - right.evaluate()
            elif root == 'times':
                return left.evaluate() * right.evaluate()
            elif root == 'divide':
                return left.evaluate() / right.evaluate()
            elif root == 'intdivide':
                return left.evaluate() // right.evaluate()
            elif root == 'power':
                return left.evaluate() ** right.evaluate()
            elif root == 'mod':
                return left.evaluate() % right.evaluate()      

In [62]:
expr1 = ExpressionTree('x')
expr2 = ExpressionTree(1)
expr3 = ExpressionTree('times', expr1, expr2)
expr3.infixForm()                     

'x'

In [75]:
expr1 = ExpressionTree('x')
expr2 = ExpressionTree(1)
expr3 = ExpressionTree('times', expr1, expr2)
expr4 = ExpressionTree('a')
expr5 = ExpressionTree(0)
expr3.contents = ('plus', expr4, expr5)
expr3.infixForm()

'(a) + (0)'

In [5]:
expr3.contents = ('power',expr1,expr2)

In [6]:
expr3.infixForm()

'(5) ** (1)'

In [86]:
# cell for the new class, complete with the infixForm,
# subs and evaluate methods
class ExpressionTree(OperatorExpression):
    "A compound tree built from OperatorExpressions"
    
    def __init__(self, root, left=None, right=None):
        self.contents = (root, left, right)
    
    @property
    def contents(self):
        return self.__contents
    
    @contents.setter
    def contents(self, value):
        root, left, right = value
        if left==None or right==None:
            # default: childless tree, contents consist of a 1-tuple
            self.__contents = (root,)
        elif (
            root=='plus' and right.contents==(0,) or
            root=='subtract' and right.contents==(0,) or
            root=='times' and right.contents==(1,) or
            root=='power' and right.contents==(1,) or
            root=='intdivide' and right.contents==(1,)
        ):
            # simplify to left contents
            self.__contents = left.contents
        elif (
            root=='plus' and left==0 or
            root=='times' and left==1
        ):
            # simplify to right contents
            self.__contents = right.contents
        elif root=='power' and left==1:
            # simplify to 1
            self.__contents = (1,)
        else:
            # parent tree, contents consist of a 3-tuple
            self.__contents = (root, left, right)

    def prefixForm(self):
        "Method: returns string consisting of nested expressions"
        if len(self.contents) == 1:
            # childless tree: return the root node as a string
            return str(self.contents[0])
        else:
            # parent tree: recurse down the structure
            root, left, right = self.contents[0], self.contents[1], \
self.contents[2]
            return \
        root+'('+left.prefixForm()+', '+right.prefixForm()+')'       

    def infixForm(self):
        "Method: returns string consisting of nested expressions in infix form"
        if len(self.contents) == 1:
            return str(self.contents[0])
        else:
            root, left, right = self.contents[0], self.contents[1], \
self.contents[2]
            return '('+left.infixForm()+') '+self.infixOperator()+' ('+\
right.infixForm()+')' 
        
    def subs(self, var, val):
        """Method: returns an ExpressionTree 
        with var substituted for val"""
        if len(self.contents) == 1:
            if self.contents[0] == var:
                return ExpressionTree(val)
            else:
                return self
        else:
            root, left, right = self.contents[0], self.contents[1], \
self.contents[2]
            return ExpressionTree(root, left.subs(var, val), \
right.subs(var, val))

    def evaluate(self):
        "Method: returns the numerical value of an ExpressionTree"
        if len(self.contents) == 1:
            if isinstance(self.contents[0], str):
                return eval(self.contents[0])
            else:
                return self.contents[0]
        else:
            root, left, right = self.contents[0], self.contents[1], \
self.contents[2]
            if root == 'plus':
                return left.evaluate() + right.evaluate()
            elif root == 'subtract':
                return left.evaluate() - right.evaluate()
            elif root == 'times':
                return left.evaluate() * right.evaluate()
            elif root == 'divide':
                return left.evaluate() / right.evaluate()
            elif root == 'intdivide':
                return left.evaluate() // right.evaluate()
            elif root == 'power':
                return left.evaluate() ** right.evaluate()
            elif root == 'mod':
                return left.evaluate() % right.evaluate()    

In [87]:
expr1 = ExpressionTree('x')
expr2 = ExpressionTree(1)
expr3 = ExpressionTree('times', expr1, expr2)
expr3.infixForm()

'x'

In [88]:
expr1 = ExpressionTree('x')
expr2 = ExpressionTree(1)
expr3 = ExpressionTree('times', expr1, expr2)
expr4 = ExpressionTree('a')
expr5 = ExpressionTree(0)
expr3.contents = ('plus', expr4, expr5)
expr3.infixForm()

'a'

In [84]:
expr3.contents = ('power',expr1,expr2)

In [85]:
expr3.infixForm()

'5'

## Instance, class and static methods

In [12]:
class Rational(object):
    "Rational number as a 2-tuple of ints"
    
    def __init__(self, a, b):
        self.numden = (a, b)
    
    def num(self):
        return self.numden[0]
    
    def den(self):
        return self.numden[1]

In [13]:
rat1 = Rational(3,4)

In [15]:
rat1.den()

4

In [91]:
class Rational(object):
    
    def __init__(self, a, b):
        self.numden = (a, b)
    
    @property
    def numden(self):
        return self.__numden
    
    @numden.setter
    def numden(self, value):
        from math import gcd
        a, b = value
        h = gcd(abs(a),abs(b))
        s = b//abs(b)
        self.__numden = (a*s//h,b*s//h)
    
    def num(self):
        return self.numden[0]
    
    def den(self):
        return self.numden[1]

In [92]:
rat1 = Rational(6,8)
print(rat1.numden)
rat2 = Rational(3,-4)
print(rat2.numden)

(3, 4)
(-3, 4)


In [20]:
class Rational(object):
    
    def __init__(self, a, b):
        self.numden = (a, b)
    
    @property
    def numden(self):
        return self.__numden
    
    @numden.setter
    def numden(self, value):
        a, b = value
        h = Rational.gcd(abs(a),abs(b))
        s = b//abs(b)
        self.__numden = (a*s//h,b*s//h)
    
    def num(self):
        return self.numden[0]
    
    def den(self):
        return self.numden[1]
    
    @staticmethod
    def gcd(a, b):
        while b>0:
            a, b = b, a%b
        return a

In [22]:
rat1 = Rational(6,8)

In [25]:
rat1.numden

(3, 4)

In [24]:
rat1 = Rational(-6,-8)

In [93]:
class Rational(object):
    
    def __init__(self, a, b):
        self.numden = (a, b)
    
    @property
    def numden(self):
        return self.__numden
    
    @numden.setter
    def numden(self, value):
        a, b = value
        h = Rational.gcd(abs(a),abs(b))
        s = b//abs(b)
        self.__numden = (a*s//h,b*s//h)
    
    def num(self):
        return self.numden[0]
    
    def den(self):
        return self.numden[1]
    
    @staticmethod
    def gcd(a, b):
        while b>0:
            a, b = b, a%b
        return a
    
    def minus(self):
        a, b = self.numden
        return self.__class__(-a, b)   
    
    def reciprocal(self):
        a, b = self.numden
        return self.__class__(b, a)        
    
    @classmethod
    def times(cls, rat1, rat2):
        a1, b1 = rat1.numden
        a2, b2 = rat2.numden
        return cls(a1*a2,b1*b2)
    
    @classmethod
    def plus(cls, rat1, rat2):
        a1, b1 = rat1.numden
        a2, b2 = rat2.numden
        return cls(a1*b2+a2*b1,b1*b2)
    
    @classmethod
    def divide(cls, rat1, rat2):
        return cls.times(rat1, rat2.reciprocal())
    
    @classmethod
    def subtract(cls, rat1, rat2):
        return cls.plus(rat1, rat2.minus())

In [97]:
rat1 = Rational(3,4)
rat2 = Rational(2,3)
print(Rational.times(rat1,rat2).numden)
print(Rational.plus(rat1,rat2).numden)
print(Rational.divide(rat1,rat2).numden)
print(Rational.subtract(rat1,rat2).numden)

(1, 2)
(17, 12)
(9, 8)
(1, 12)
